# QAOA for the QAP

In [ ]:
#INITIALISATION CELL
import qiskit
from qiskit import Aer
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua.components.optimizers import SPSA, COBYLA
from qiskit.circuit.library import RealAmplitudes,TwoLocal #(uses CX entangling) 
from qiskit.aqua import QuantumInstance

from docplex.mp.model import Model
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.applications.ising import docplex

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
# set_qiskit_aqua_logging(logging.DEBUG)  # choose INFO, DEBUG to see the log

# useful additional packages 
%matplotlib inline
import numpy as np
import pandas as pd
import math

Important prelim functions

In [ ]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = []
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = []
            for i in string2:
                if i != '':
                    emptyarray.append(int(i))
            matrix.append(emptyarray)
        npmatrix = np.array(matrix) 
        return npmatrix
    return string_to_integers(final_data1),string_to_integers(final_data2)

In [ ]:
def qap_value(z, MatrixLoc, MatrixFlow):
    """
    Input:
    z (list[int]): list of allocations
    MatrixLoc (numpy array): matrix of distances
    MatrixFlow (numpy array): matrix of flow
    
    Output:
    float: value of the QAP
    """
    matrix_length = len(MatrixLoc)
    x = np.reshape(z, (matrix_length,matrix_length))
    
    total = 0
    for i in range(matrix_length):
        for j in range(matrix_length):
            for k in range(matrix_length):
                for p in range(matrix_length):
                        total += MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p]
    
    return total

Feasibility functions

In [ ]:
def qap_feasible(x):
    """
    Input:
    x (numpy.ndarray) : binary string as numpy array.

    Output:
    bool: feasible or not.
    """
    n = int(np.sqrt(len(x)))
    y = np.reshape(x, (n,n))
   
    for i in range(n):
        if sum(y[i, p] for p in range(n)) != 1:
            return False
    for p__ in range(n):
        if sum(y[i, p__] for i in range(n)) != 1:
            return False
    return True

In [ ]:
def choose_best_feasible(eigenstates):
    """
    Input:
    eigenstates = dictionary
    
    Output:
    feasible binary 1D numpyarray
    probability of this answer
    """
    bestinarray = sorted(eigenstates.items(), key=lambda item: item[1])[::-1]
    feasible = False
    counter = 0
    total = sum(eigenstates.values())
    
    feasible=False
    while feasible==False and counter<len(bestinarray):
        #string to array
        bestasint = np.array([0])
        for i in bestinarray[counter][0]:
            bestasint = np.hstack((bestasint, int(i)))
        feasible = qap_feasible(bestasint[1:])
        frequency = bestinarray[counter][1]
        counter += 1
        
    if feasible == False:
        return feasible
    else:
        return bestasint[1:], frequency/total

## The QAOA function

In [ ]:
def testing_quantum(machine, ins, SPtrial, SHOT, optimal_point):
    """
    Input:
    machine: the name of available machine
    ins: the QAP instance
    SPtrial: max_trials for SPSA 
    SHOT: number of shots
    optimal_point: numpy array of optimal points from simulator
    
    Output:
    outmatrix: 30 * [eigensolution value, time, feasibility]
    (outfile "thirty_trials-<machine name>-<instance name>.csv")
    """
    
    #get matrix
    datamatrix = CSVtoNumpyArray(ins)
    MatrixLoc = datamatrix[0]
    MatrixFlow = datamatrix[1]
    n = len(MatrixLoc)

    # Create an instance of a model and variables.
    thename = "qap" + str(n)
    mdl = Model(name=thename)
    x = {(i,p): mdl.binary_var(name='x_{0}_{1}'.format(i,p)) for i in range(n) for p in range(n)}

    # Object function
    qap_func = mdl.sum(MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p] 
                       for i in range(n) for j in range(n) for p in range(n) for k in range(n))
    mdl.minimize(qap_func)

    # Constraints
    for i in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for p in range(n)) == 1)
    for p in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for i in range(n)) == 1)
    print(mdl.export_to_string())
    qubitOp_docplex, offset_docplex = docplex.get_operator(mdl) 

    #Setup QAOA
    seed = 10598
    spsa = SPSA(maxiter=SPtrial)
    print('num of qubits; ', qubitOp_docplex.num_qubits)
    
    #30 trials
    file = open("thirty_trials-" + str(machine) + "-" + str(ins) ,"w")
    file.write("value,   feasible,   frequency, time, iteration" + "\n")
    ans = np.zeros(5)
    for i in range(30):
        try:
            #backend = Aer.get_backend(machine)
            backend = provider.get_backend(machine)
            quantum_instance = QuantumInstance(backend=backend,seed_simulator=seed, seed_transpiler=seed, 
                                               skip_qobj_validation = False, shots = SHOT)
            qaoa = QAOA(qubitOp_docplex,optimizer=spsa, p=j, quantum_instance=quantum_instance,include_custom=True)
            result = qaoa.run(quantum_instance)

            #Output processing
            print(' Eigenstate:', result['eigenstate'])
            print('QAOA time:', result['optimizer_time'])
            n = len(list(result['eigenstate'].values()))
            solution = np.hstack((np.array(list(result['eigenstate'].values())).reshape(n,1),
                                  np.array(list(result['eigenstate'].keys())).reshape(n,1)))
            print(solution)
            for r in solution:
                file.write(str(qap_value(np.array(list(r[1])),MatrixFlow,MatrixLoc)) + "," + 
                           str(qap_feasible(np.array(list(r[1])))) + "," + str(r[0]) + ","+ 
                           str(result['optimizer_time']) + "," + str(i) + "\n")
                ans = np.vstack((ans,np.array([qap_value(np.array(list(r[1])),MatrixFlow,MatrixLoc),
                                               qap_feasible(np.array(list(r[1]))),float(r[0]),result['optimizer_time'],i])))
        except:
            file.write("An error occurred on iteration " + str(i) + " of the 30 trials")
            print("AN ERROR OCCURRED ON ITERATION" + str(i) + " of the 30 trials")
            ans = np.vstack((ans,np.array([math.inf,False,0,np.nan,i])))
        print("Iteration "+ str(i) + " is complete.")
    file.close()
    return ans[1:] 

In [ ]:
def read_optimal(ins):
    data = pd.read_csv("initial_point-" + str(ins),header = None)
    n = len(data[0]) - 1
    ans = []
    data[0][0] = data[0][0][1:]
    data[0][n] = data[0][n][:-1]
    for i in data[0]:
        r = i.split(" ")
        for t in r:
            if t!='':
                ans.append(float(t))
    return np.array(ans)

# The main cells

These are the cells running solutions

In [ ]:
#3 by 3 thirty trials

import logging
logging.basicConfig(level=logging.DEBUG) # log the steps of the algorithm and results
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
machines = 'ibmq_cambridge'
instances = "made3.csv"
optimal_point = read_optimal(instances)
ans = testing_quantum(machines, instances, 1, 1024,optimal_point)

In [ ]:
#4 by 4 thirty trials

import logging
logging.basicConfig(level=logging.DEBUG) # log the steps of the algorithm and results
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
machines = 'ibmq_cambridge'
instances = "made4.csv"
optimal_point = read_optimal(instances)
ans4 = testing_quantum(machines, instances, 1, 1024,optimal_point)

In [ ]:
qiskit.__qiskit_version__